In [ ]:
!pip install gradientai --upgrade
!pip install rake-nltk
from gradientai import Gradient
import os
import spacy
import requests
import nltk
from rake_nltk import Rake

# Setting Environment Variables for GRADIENT
os.environ['GRADIENT_ACCESS_TOKEN'] = ''
os.environ['GRADIENT_WORKSPACE_ID'] = ''

# Initialize the Gradient SDK
gradient = Gradient()

# Initialize NLP tools
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('stopwords')
r = Rake()

try:
    base_model = gradient.get_base_model(base_model_slug="llama2-7b-chat")
    new_model_adapter = base_model.create_model_adapter(name="test model 3")
    print(f"Created model adapter with id {new_model_adapter.id}")
except Exception as e:
    print("Failed to initialize the model:", e)
    exit()

class ContextualChatbot:
    def __init__(self, model_adapter, learning_scores, personalized=True):
        self.model_adapter = model_adapter
        self.learning_scores = learning_scores
        self.personalized = personalized
        self.conversation_history = []  # Use a list for context management

    def toggle_personalization(self):
        self.personalized = not self.personalized
        print(f"Personalization {'enabled' if self.personalized else 'disabled'}.")
        if self.personalized:
            print("Personalized responses are created using your learning scores to make interactions more relevant.")
        else:
            print("Non-personalized responses do not use your specific learning data.")

    def generate_response(self, user_prompt):
        self.conversation_history.append(f"User: {user_prompt}")

        if self.personalized:
            context = "Using your learning scores to enhance our conversation."
            self.conversation_history.append(context)

        # Limit context length to maintain efficiency
        max_context_length = 10
        if len(self.conversation_history) > max_context_length:
            self.conversation_history = self.conversation_history[-max_context_length:]

        response = self.model_adapter.complete(query="\n".join(self.conversation_history), max_generated_token_count=511).generated_output
        self.conversation_history.append(f"Assistant: {response}")

        return response

    def extract_topic(self, user_prompt):
        # Extract topic using NER and keyword extraction
        doc = nlp(user_prompt)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        keywords = r.extract_keywords_from_text(user_prompt)
        # Combine entities and keywords for a more comprehensive topic
        if entities is None or keywords is None:
            print("Warning: NER or keyword extraction failed to identify any entities or keywords.")
            topic = ""  # Or set a default topic
        else:
            topic = " ".join(entities + keywords)

        return topic


chatbot = ContextualChatbot(model_adapter=new_model_adapter, learning_scores=learning_scores)

# Chat loop
print("Welcome to the chatbot!")
print("Your learning measures scores:", learning_scores)
print("Type 'quit' to exit, or 'toggle' to switch personalization.")
while True:
    user_prompt = input("\nPlease enter your query or question: ")

    if user_prompt.lower() in ["quit", "exit", "q"]:
        print("Exiting the chat. Goodbye!")
        break
    elif user_prompt.lower() == "toggle":
        chatbot.toggle_personalization()
        continue

    response = chatbot.generate_response(user_prompt)

    # Extract topic and fetch academic content
    topic = chatbot.extract_topic(user_prompt)

    response_type = "Personalized" if chatbot.personalized else "Non-Personalized"
    print(f"\n{response_type} Response:", response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Created model adapter with id 955876af-c137-43c5-8ae5-f4764587fdb0_model_adapter
Welcome to the chatbot!
Your learning measures scores: {'Conscientiousness': 5.16, 'Motivation': 6.17, 'Understanding': 5.98, 'Engagement': 6.17}
Type 'quit' to exit, or 'toggle' to switch personalization.

Please enter your query or question: How to improve my reading skills

Personalized Response:  Great to hear that you're interested in improving your reading skills! Based on your learning scores, I can provide you with some personalized tips and strategies to help you enhance your reading abilities.

1. Set specific goals: Identify what you want to achieve through reading, whether it's to improve your vocabulary, comprehend complex texts, or simply to enjoy reading more. Setting specific goals will help you stay motivated and focused.
2. Develop a reading routine: Establish a regular time and place for reading, and try to make it a habit. This could be reading for 15-20 minutes each day, or dedicating 